### 박스오피스 영화 스크롤링 및 분석

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd

# ------------------------------------------------------
# 0) 분석
# ------------------------------------------------------
# ## 첫번째 영화 제목의 주소
# #main_pack > div.sc_new.cs_common_module.case_list.color_5._au_movie_list_content_wrap > div.cm_content_wrap > div > div > div.mflick > div._panel_popular._tab_content > div.list_image_info.type_pure_top > div > ul:nth-child(1) > li:nth-child(1) > a > div > div.title_box > strong
# -> '#main_pack div.list_image_info.type_pure_top ul'  : 이렇게 정리해야함



# ------------------------------------------------------
# 1) 현재 상영 영화 페이지 불러오기
# ------------------------------------------------------
## 주소 불러와서 urlopen하는건 encoding 에러 발생해서 다른 방법으로 해결
url = "https://movie.naver.com/movie/running/current.naver"
req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
html = urlopen(req).read().decode("utf-8", "ignore")
htmlBS = BeautifulSoup(html, "html.parser")

htmlBS




## 다음으로 클릭하는 버튼




<!DOCTYPE html>
 <html lang="ko"><head> <meta charset="utf-8"/> <meta content="strict-origin-when-cross-origin" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="현재상영영화 : 네이버 검색" property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'현재상영영화'의 네이버 검색 결과입니다." property="og:description"/> <meta content="'현재상영영화'의 네이버 검색 결과입니다." lang="ko" name="description"/> <title>현재상영영화 : 네이버 검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_32x32_240820.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_251023.css" rel="stylesheet" type="text/css"/> <link href=

In [ ]:
# ------------------------------------------------------
# 2) 영화 정보 파싱
# ------------------------------------------------------
def parse_naver_current_movies(htmlBS):
    movies = []

    # card_item 하나가 영화 하나
    cards = htmlBS.select("div.card_item") ## 영화 자체 하나가 card_item안에 들어있음

    for card in cards: ## 영화 자체를 하나씩 가져와서 확인

        # 제목
        ## 제목이 들어있는 a태그를 찾아라
        title_tag = card.select_one("div.title.multi_line._ellipsis a") ## 여기 안에 제목 들어있음
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True) ## 안에있는 텍스틑 뽑기

        # 장르
        genre = None
        overview_dl = card.select_one("dl.info_group") ## 여기 안에 개요 들어있음
        if overview_dl: ## 값이 있으면
            dd_tags = overview_dl.select("dd") # /dd 선택
            if len(dd_tags) >= 1: ## 장르랑 러닝타임이 같이 들어와서, 장르만 선택
                genre = dd_tags[0].get_text(strip=True)

        # 개봉일
        release_date = None
        info_groups = card.select("dl.info_group")
        if len(info_groups) >= 2: # 안에 개요랑 개봉일이 있음. 2번을 선택하기
            release_dl = info_groups[1]   # 2번째 dl
            dd = release_dl.select_one("dd")
            if dd:
                release_date = dd.get_text(strip=True)

        # 평점
        rating = None
        # dl.info_group.type_visible 안에서 </span> 태그안에 클래스 num안에 평점이 있음
        # -> 띄워쓰기는 하위자손 이라는 뜻 / 태그명.클래스명
        rating_tag = card.select_one("dl.info_group.type_visible span.num") 
        if rating_tag:
            try:
                ## object라서 float로 변경
                rating = float(rating_tag.get_text(strip=True))
            except:
                rating = None

        ## 값 다 추가하기
        movies.append({
            "제목": title,
            "장르(개요)": genre,
            "평점": rating,
            "개봉일": release_date,
        })
    print(movies)
    return pd.DataFrame(movies)

In [117]:
# ------------------------------------------------------
# 평점순으로 나열하기
# ------------------------------------------------------
df = parse_naver_current_movies(htmlBS)
display(df.sort_values('평점', ascending=False))


[{'제목': '위키드: 포 굿', '장르(개요)': '판타지', '평점': 7.55, '개봉일': '2025.11.19.'}, {'제목': '나우 유 씨 미 3', '장르(개요)': '범죄', '평점': 7.83, '개봉일': '2025.11.12.'}, {'제목': '극장판 체인소 맨: 레제편', '장르(개요)': '애니메이션', '평점': 9.27, '개봉일': '2025.09.24.'}, {'제목': '나혼자 프린스', '장르(개요)': '코미디', '평점': 7.53, '개봉일': '2025.11.19.'}, {'제목': '국보', '장르(개요)': '드라마', '평점': 9.1, '개봉일': '2025.11.19.'}, {'제목': '프레데터: 죽음의 땅', '장르(개요)': 'SF', '평점': 8.62, '개봉일': '2025.11.05.'}, {'제목': '퍼스트 라이드', '장르(개요)': '코미디', '평점': 7.57, '개봉일': '2025.10.29.'}, {'제목': '8번 출구', '장르(개요)': '스릴러', '평점': 8.36, '개봉일': '2025.10.22.'}]


,제목,장르(개요),평점,개봉일
2,극장판 체인소 맨: 레제편,애니메이션,9.27,2025.09.24.
4,국보,드라마,9.10,2025.11.19.
5,프레데터: 죽음의 땅,SF,8.62,2025.11.05.
7,8번 출구,스릴러,8.36,2025.10.22.
1,나우 유 씨 미 3,범죄,7.83,2025.11.12.
6,퍼스트 라이드,코미디,7.57,2025.10.29.
0,위키드: 포 굿,판타지,7.55,2025.11.19.
3,나혼자 프린스,코미디,7.53,2025.11.19.


In [ ]:
# ------------------------------------------------------
# 장르별 영화 수 / 평균 평점
# ------------------------------------------------------
## 장르별 개수 확인하는 코드
# print("\n=== 장르(개요) 분포 ===")
# print(df["장르(개요)"].value_counts())

def genre_analysis(df: pd.DataFrame):
    # 장르가 비어있지 않은 것만
    valid = df[df["장르(개요)"] != ""].copy()

    # 장르별 개수와 평균 평점
    genre_group = valid.groupby("장르(개요)")["평점"].agg(["count", "mean"]).reset_index()
    genre_group = genre_group.rename(columns={
        "장르(개요)": "장르",
        "count": "영화편수",
        "mean": "평균평점"
    }).sort_values("평균평점", ascending=False)

    print("=== 장르별 영화 수 / 평균 평점 ===")
    display(genre_group)

    return genre_group


genre_stats = genre_analysis(df)


=== 장르별 영화 수 / 평균 평점 ===


,장르,영화편수,평균평점
4,애니메이션,1,9.27
1,드라마,1,9.10
0,SF,1,8.62
3,스릴러,1,8.36
2,범죄,1,7.84
5,코미디,2,7.55
6,판타지,1,7.54


## 인사이트 도출

높은 평점을 받은 장르: 애니메이션·드라마

- 둘 다 9점대 초반으로, 현재 상영작 중 관객 만족도가 가장 높은 장르.

- 특히 애니메이션은 1편이지만, 최고 평점(9.27)을 기록 → "애니메이션에 대한 충성도가 높은걸로 보임"

코미디는 편수는 많지만 평균 평점은 상대적으로 낮음 (7.55, 2편)

- 웃음 코드가 취향을 많이 타고, 작품별 완성도 차이가 있기 때문에
대중적으로는 무난하지만 "압도적 호평"까지는 잘 안 가는 장르.

판타지 장르도 7점대 중반으로 중간 수준

- 시각적 볼거리·세계관은 강점이지만,
스토리 호불호 때문에 평점이 중간~중상 정도에 머무는 경향을 보임.


## 한계점 

샘플의 한계

- 대부분의 장르가 1편만 있는게 아쉬움

- 장르 전체의 성향이라기보다는 현재 상영 중인 대표 작품의 평가를 그대로 반영한 수치입니다.
